In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np 
import math
import time 
from scipy import linalg as la
import copy as cp 

In [4]:
'''
this cell sets up the matrix for non-interacting particles. 
'''

n = 200

#(p0,pmax,n)
p = np.linspace(0,10, num = n)


#uses of stepsize are actually defined in terms of 1/h for simplicity
hh = (n/(p[-1] - p[0]))**2
hh2 = 2*hh


#this cuts off the endpoints 
A = np.zeros(shape = (n-1,n-1))
V = np.zeros(n+1)


for i in range(n):
    V[i] = p[i]**2

for i in range(n-1):
        A[i][i] = hh2+V[i+1]
        
#we evaluate v[i+1] so that we avoid the 0 point 
#dont overwrite (n-2)
for i in range(n-2):
        A[i+1][i]= A[i][i+1] = -hh



#test that the matrix is set up correctly, expect 3,7,11,...
evals = la.eigvals(A)


s = np.sort(evals)
print(s[0:4])



[  3.01428603+0.j   7.03122801+0.j  11.04564063+0.j  15.05752148+0.j]


In [6]:

#this is essentially the same as above but re-writen for coulomb interaction assuming omega = 1 
#i have removed redundant comments 

n=200
p = np.linspace(0,10, num = n)

hh = (n/(p[-1] - p[0]))**2
hh2 = 2*hh

A = np.zeros(shape = (n-1,n-1))
V = np.zeros(n+1)



for i in range(n):
    V[i] = (p[i]**2)+(p[i]**(-1))

for i in range(n-1):
        A[i][i] = hh2+V[i+1]

for i in range(n-2):
        A[i+1][i]= A[i][i+1] = -hh

        
        
evals = la.eigvals(A)
s = np.sort(evals)
print(s[0:4])


[  4.06995310+0.j   7.93900655+0.j  11.86328232+0.j  15.81224591+0.j]


/home/mroosa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: RuntimeWarning: divide by zero encountered in double_scalars


In [12]:
test = np.array([[2.0,-4.0],[-4.0,1.0]])

#find largest non-diagonal and its address 
def pickmax(a):
    
    #initialize the maximum value and its indices, big=-1 to gaurentee the > is met at least once
    big = -1. 
    indi = 0
    indj = 0
    
    #iterate over matrix (input a) to find largest value, i+1 in j to get i != j w/o extra if statement
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if abs(a[i][j])> big:
                big = abs(a[i][j])
                indi = i
                indj = j
    return big, indi, indj


pickmax(test)

(4.0, 0, 1)

In [13]:
test = np.array([[2.0,-4.0],[-4.0,1.0]])

#set the sin,cos and tan for the max value
def settrig(a,p,q):
    
    #initialize tan, sin,l and cos respectively  
    t = s = c = 0
    
    #handles the non-singular case
    if a[p][q] != 0:
        tau = (a[q][q]-a[p][p])/(2*a[p][q])
    
        if tau < 0:
             t = -(-tau+math.sqrt(1+tau**2))**(-1.)
        else: 
            t = (tau+math.sqrt(1+tau**2))**(-1.)
        c = (math.sqrt(1+t**2))**(-1.)
        s = t*c

    else:
        c = 1.0
        s = 0.0
    return t,c,s


settrig(test,0,1)

(0.88278221853731875, 0.7496781758158659, 0.66180256323574027)

In [15]:
test = np.array([[2.0,-4.0],[-4.0,1.0]])
testvals = la.eigvals(test)



# 1 jacobi transformation
def jacobi(a):
    
    #find inputs to jacobi algo 
    maxval, p, q = pickmax(a)
    t,c,s = settrig(a,p,q)
    
    #note effective 'call by value' 
    b = cp.copy(a)
    
    #jacobi iteration 
    for i in range(len(a)):
        if i!=p or i!=q:
            #rotation 
            b[i][p] = a[i][p]*c-a[i][q]*s
            b[i][q] = a[i][q]*c+a[i][p]*s
            #force symmetry
            b[p][i] = b[i][p]
            b[q][i] = b[i][q]
    #force realtions for diagonal vals & maxval
    b[p][p] = a[p][p]*c**2-2*a[p][q]*c*s+a[q][q]*s**2
    b[q][q] = a[p][p]*s**2+2*a[p][q]*c*s+a[q][q]*c**2
    b[p][q] = b[q][p] = 0
    
    return b,maxval
    
jacobi(test)

(array([[ 5.53112887,  0.        ],
        [ 0.        , -2.53112887]]), 4.0)

In [17]:
test3 = np.array([[4.,2.,1.],[2.,4.,1.],[1.,1.,4.]])


#Actual solver input: nxn matrix, value of tolerance (suggested 10**-5) 
def esolver(matrix, tolerance):
    
    #initailize & gaurentee 1  iteration 
    maxval = tolerance+1
    
    #iterate until tolerable 
    while tolerance<=maxval:
        matrix, maxval =jacobi(matrix )
        
    return matrix



esolver(test3,.001)

array([[ 2.        ,  0.        ,  0.        ],
       [ 0.        ,  6.73205081,  0.        ],
       [ 0.        ,  0.        ,  3.26794919]])